# Predict Images
Given a model and images the model is applied and the result is added to the metadata

In [ ]:
!pip3 install tensorflow==2.4.0 wget==3.2

In [ ]:
import wget
wget.download(
    'https://raw.githubusercontent.com/IBM/claimed/master/component-library/claimed_utils.py'
)
from claimed_utils import parse_args_to_parameters
import tensorflow as tf
from tensorflow import keras
from claimed_utils import unzip
import os
import os.path
import glob
import pandas as pd
import numpy as np

In [ ]:
# Column name containing the target/prediction value (the real measured value)
target_column = os.environ.get('target_column', 'target')

# Column name containing the image file name
image_column = os.environ.get('image_column', 'filename')

# Target image shape (the model expects)
image_shape = os.environ.get('image_shape', '400,400')

# Column name under which the prediction has to be stored
prediction_column = os.environ.get('prediction_column', 'prediction')

# data CSV file containing meta information about the images (e.g. real class, ...)
metadata = os.environ.get('metadata', 'metadata.csv')

# data CSV file containing updated meta information about the images with "prediction_column" added
output_metadata = os.environ.get('output_metadata', 'metadata.csv')

# model zip file name - currently only zipped TensorFlow 2.x pb files supported
model_zip = os.environ.get('model_zip', 'model.zip')

# zip file name containing all the images
data_zip = os.environ.get('data_zip', 'data.zip')

parse_args_to_parameters()

In [ ]:
unzip('.', model_zip)
unzip('.', data_zip)

In [ ]:
model = keras.models.load_model('model')

In [ ]:
classes = glob.glob("data/*")
classes = map(lambda s: s.split('/')[1], classes)
classes = list(classes)
classes

In [ ]:
df = pd.read_csv(metadata)

In [ ]:
df

In [ ]:
dataset = []

for i, row in df.iterrows():
    target_size = 'dummy'  # make the compiler happy
    exec('target_size = (' + image_shape + ')')
    image = tf.keras.preprocessing.image.load_img(
        'data/' + row[target_column] + '/' + row[image_column],
        target_size=target_size
    )
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    predictions = model.predict(input_arr)
    prediction = predictions[0]
    prediction = classes[np.argmax(prediction)]
    dataset.append(prediction)

In [ ]:
dataset = np.array(dataset)

In [ ]:
df = pd.read_csv(metadata)

In [ ]:
df[prediction_column] = dataset.tolist()

In [ ]:
df

In [ ]:
df.to_csv(output_metadata, index=False)